In [1]:
# %load_ext pycodestyle_magic
# %flake8_on
# import logging
# logging.getLogger("flake8").setLevel(logging.FATAL)

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Citzen Science Notebook</b> <br>
Contact author: Sreevani Jarugula <br>
Last verified to run: 2023-04-20 <br>
LSST Science Pipelines version: Weekly 2023_xx <br>
Container size: medium <br>


**Description:**
Query and send variable star images and light curves from RSP to Zooniverse

**Skills:** Use various TAP tables, including joining multiple tables. Get calexp images. Extract time-series photometry.

**LSST Data Products:** TAP tables dp02_dc2_catalogs.MatchesTruth, TruthSummary, ForcedSource, CcdVisit<br>

**Packages:** astropy, lsst.rsp.get_tap_service, lsst.rsp.retrieve_query, lsst.daf.butler, lsst.afw.display, lsst.geom 

**Credit:** Tutorial notebooks 03a, 04a, 04b, 07b, and 08

**Support:** Support is available and questions are welcome - (some email/link etc)

DEBUG VERSION note that this version of the notebook contains additional debugging and the first cell will need to be run once

## 1. Introduction

This notebook is intended to guide a PI through the process of sending variable data from the Rubin Science Platform (RSP) to the Zooniverse. A detailed guide to Citizen Science projects, outlining the process, requirements and support available is here: (link to citscipiguide) The data sent can be currated on the RSP as a necessary and take many forms. Here, we include an example of sending png cutout images of a single variable star over different exposures. We encourage PIs new to the Rubin dataset to explore the tutorial notebooks and documentation.

This notebook will restrict the number of object sent to the Zooniverse to one variable star over five exposures.

### Log in to the Zooniverse Platform & Activate Citizen Science SDK

If you haven't already, create a Zooniverse account here. and create your project. Your project must be set to "public". To set your project to public, select the "Visibility" tab. Note you will need to enter your username, password, and project slug below.

After creating your account and project, return to this notebook.

Supply your email and project slug below.

A "slug" is the string of your Zooniverse username and your project name without the leading forward slash, for instance: "username/project-name".

For more details, see: https://www.zooniverse.org/talk/18/967061?comment=1898157&page=1.

IMPORTANT: Your Zooniverse project must be set to "public", a "private" project will not work. Select this setting under the "Visibility" tab, (it does not need to be set to live). The following code will not work if you have not authenticated in the cell titled "Log in to Zooniverse".

In [1]:
# %load_ext pycodestyle_magic
# %flake8_on
# import logging
# logging.getLogger("flake8").setLevel(logging.FATAL)
email = "jsv1206@gmail.com"  
slugName = "sreevani/test-project-sj" 
%run Citizen_Science_SDK.ipynb

Installing external dependencies...
Done installing external dependencies!
Enter your Zooniverse credentials...


Username:  sreevani
 ········


You now are logged in to the Zooniverse platform.
Loaded Citizen Science SDK


## 1.1 Package imports

In [2]:
from utils import *

### 1.1.1 Initializing TAP and Butler

In [3]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
service, butler, skymap = setup_butler(config, collection)

### 1.2 Define Functions and Parameters

### 1.2.1 Variable star related parameters

In [4]:
query_num_stars = 2 # number of variable stars to query 

# Set any RA and DEC in degrees. This is the centre of your search radius
# In this example notebook we are looking at one known RR-Lyrae. 

ra_known_rrl = 62.1479031
dec_known_rrl = -35.799138

search_radius = 0.001 # Search radius in degrees

num_variable_images = 5 # For each variable stars, number of images to query to create flipbook (gif)

image_size = 100 # image size for cutouts

bands = ['g','r','i'] # bands to get flux for to create a lightcurve

### 1.2.4 Create relevant directories

In [5]:
plots = [] # empty list for plots

# main directory
batch_dir = './variable_stars_output/' 

if os.path.isdir(batch_dir) == False:
    os.mkdir(batch_dir)

# cutputs directory
if os.path.isdir(batch_dir+'images') == False:
    os.mkdir(batch_dir+'images')
else:
    os.system('rm -r '+batch_dir+'images/*')

# light curve directory
if os.path.isdir(batch_dir+'lc_plots') == False:
    os.mkdir(batch_dir+'lc_plots')
else:
    os.system('rm -r '+batch_dir+'lc_plots/*')

# light curve text file directory
if os.path.isdir(batch_dir+'text_files') == False:
    os.mkdir(batch_dir+'text_files')
else:
    os.system('rm -r '+batch_dir+'text_files/*')

### 1.2.5 Query and Plotting functions

In [6]:
def query_stars(ra_deg, dec_deg, radius_deg, limit):
    """
    Query variable stars from dp02_dc2_catalogs.MatchesTruth and dc2_catalogs.TruthSummary
    
    To query more than one star within the circle of search, change = 1 to <= 1
    
    Selecting stars (truth_type=2)
    variable (is_variable = 1)
    is_pointsource = 1

    Input Parameters
    ----------
    ra_deg : ra of the centre of search in degrees
    dec_deg : dec of the centre of search in degrees
    radius_deg : radius within which to search for
    limit : number of variable stars to retireve
    
    Returns
    ----------
    Table of variable stars as pandas dataframe
    """
    query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec "\
            "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
            "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
            "WHERE ts.truth_type=2 "\
            "AND ts.is_variable = 1 "\
            "AND ts.is_pointsource = 1 "\
            "AND mt.match_objectId > 1 "\
            "AND CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', "+ str(ra_deg)+", "+str(dec_deg)+", "+str(radius_deg)+")) = 1 "\
            "LIMIT "+str(limit)+" "
    results = service.search(query)
    variable_stars = results.to_table().to_pandas()
    return variable_stars

def query_flux(objid):
    """
    Query to get the flux for each variable star at all bands

    Input Parameters
    ----------
    objid : Object ID of the variable star obtained from query_stars
    
    Returns
    ----------
    Table of flux of variable star 
    """
    query = "SELECT src.band, src.ccdVisitId, src.coord_ra, src.coord_dec, "\
            "src.objectId, src.psfFlux, src.psfFluxErr, "\
            "visinfo.detector, visinfo.visitId, "\
            "scisql_nanojanskyToAbMag(psfFlux) as psfMag, "\
            "visinfo.band, "\
            "visinfo.expMidptMJD "\
            "FROM dp02_dc2_catalogs.ForcedSource as src "\
            "JOIN dp02_dc2_catalogs.CcdVisit as visinfo "\
            "ON visinfo.ccdVisitId = src.ccdVisitId "\
            "WHERE src.objectId = "+str(objid)+" "
    table = service.search(query)
    flux_table = table.to_table()
    return flux_table

## 2. Query and plot the variable stars

In [7]:
%%time
variable_stars = query_stars(ra_known_rrl, dec_known_rrl, search_radius, query_num_stars)

CPU times: user 12.8 ms, sys: 6.77 ms, total: 19.5 ms
Wall time: 1.25 s


In [8]:
variable_stars

,id_truth_type,match_objectId,ra,dec
0,835714_2,1651589610221899038,62.147903,-35.799138


### 2.1 Get Calexp images and Lightcurves

How to create csv for Zooniverse flipbook

https://help.zooniverse.org/getting-started/example/#details-subject-sets-and-manifest-details-aka-what-is-a-manifesthttps://help.zooniverse.org/getting-started/example/#details-subject-sets-and-manifest-details-aka-what-is-a-manifestCreate 


In [9]:
stars_matchid = variable_stars['match_objectId'].to_numpy()
stars_ra = variable_stars['ra'].to_numpy()
stars_dec = variable_stars['dec'].to_numpy()
df_row = []
df_final = []

fields_to_add = ["sourceId", "coord_ra", "coord_dec"] # fields to add to the maifest file
cutouts = []

for j, objid in enumerate(stars_matchid):
    
    figout_data = {"sourceId": stars_matchid[j]}
    if "coord_ra" in fields_to_add:
        figout_data["coord_ra"] = stars_ra[j]
    if "coord_dec" in fields_to_add:
        figout_data["coord_dec"] = stars_dec[j]
    
    # Query the variable star flux, detector and visit information
    ccd_flux_table = query_flux(objid)
    
    # Get calexp images from Butler and plot them
    idx_images = np.round(np.linspace(0, len(ccd_flux_table) - 1, num_variable_images)).astype(int)  #randomly select 5 images for each variable star
    image_dict = {} 
    
    for i,idx in enumerate(idx_images):
        star_ra = ccd_flux_table['coord_ra'][idx]
        star_dec = ccd_flux_table['coord_dec'][idx]
        star_detector = ccd_flux_table['detector'][idx]
        star_visitid = ccd_flux_table['visitId'][idx]
        star_id = ccd_flux_table['objectId'][idx]
        star_ccdid = ccd_flux_table['ccdVisitId'][idx]
        
        calexp_image = get_cutout_image(butler, star_ra, star_dec, star_visitid, star_detector, 'r', image_size, datasetType='calexp') # only r-band images 
        figout = make_calexp_fig(calexp_image, star_ra,star_dec,batch_dir+"/images/"+str(star_id)+"_"+str(star_ccdid)+".png")
        remove_figure(figout)
        
        image_dict['location:image_'+str(i)] = str(star_id)+"_"+str(star_ccdid)+".png"
        figout_data['location:image_'+str(i)] = str(star_id)+"_"+str(star_ccdid)+".png"
        figout_data['filename'] = str(star_id)+"_"+str(star_ccdid)+".png"
        
    cutouts.append(figout_data)
    
    # manifest file
    df_star = pd.DataFrame(data = cutouts, index=[0])
    df_final.append(df_star)
    
    # flipbook data for each variable star
    df = pd.DataFrame(data = image_dict, index=[0])
    df_row.append(df)
        
    # Light curve for each variable star    
    mjd_days, mags = get_flux(ccd_flux_table)
    figout = plotlc(bands, mjd_days, mags, batch_dir+"/lc_plots/"+"lc_"+str(objid)+".png")
    remove_figure(figout) 
    df_all_bands = []
    for band in bands:
        df = pd.DataFrame(data = {'band': [band]*len(mjd_days[band]), 'mjd_days': mjd_days[band], \
                          'mags': mags[band]}, index=None)
        df_all_bands.append(df)
    
    df_final_lc = pd.concat(df_all_bands)
    outfile = batch_dir+"/text_files/"+"lc_"+str(objid)+".csv"
    df_final_lc.to_csv(outfile, index=False, sep=',')
    
df_manifest = pd.concat(df_final) # final manifest file with all variable stars
df_manifest = df_manifest.iloc[:, [4,0,1,2,3,5,6,7]]
outfile = batch_dir+"images/metadata.csv"
df_manifest.to_csv(outfile, index=False, sep=',')
    
df_flipbook = pd.concat(df_row) # flipbook with all variable stars
outfile = batch_dir+"zooniverse_flipbook_manifest.csv"
df_flipbook.to_csv(outfile, index=False, sep=',')

In [10]:
# cutout_dir = batch_dir+"images/"

In [11]:
# manifest_path = write_metadata_file(cutouts, cutout_dir)

# print("The manifest CSV file can be found at the following relative path:")
# print(manifest_path)

## 3. Send data to Zooniverse

In [12]:
cutout_dir = batch_dir+"images/"

In [13]:
subject_set_name = "variable stars flipbook" 

In [14]:
__cit_sci_data_type = _HIPS_CUTOUTS # Important: DO NOT change this value. Update - this value may be changed.
send_data(subject_set_name, cutout_dir, cutouts)

'1. Checking batch status'

'    ** Warning! - The Zooniverse client is throwing an error about a missing subject set, this can likely safely be ignored.'

'2. Zipping up all the astro cutouts - this can take a few minutes with large data sets, but unlikely more than 10 minutes.'

'3. Uploading the citizen science data'

'4. Creating a new Zooniverse subject set'

'5. Notifying the Rubin EPO Data Center of the new data, which will finish processing of the data and notify Zooniverse'

'6. Cleaning up unused subject set on the Zooniverse platform, vendor_batch_id : 113704'